In [4]:
import pandas as pd

df = pd.read_csv("final_uni_studies.csv")

esco = pd.read_csv("/content/occupations_en.csv")


In [5]:
df['matched_label_clean'] = df['matched_label'].str.lower().str.strip()
esco['preferredLabel_clean'] = esco['preferredLabel'].str.lower().str.strip()

In [7]:
import pandas as pd

isco_to_sector = {
    '11': 'Management','12': 'Public Administration','21': 'Science & Engineering',
    '22': 'Healthcare','23': 'Education & Training','24': 'Business & Administration',
    '25': 'IT & Software','26': 'Legal, Social & Cultural','31': 'Technicians & Associate Professions',
    '32': 'Health Technicians','33': 'Business & Sales Support','34': 'Legal & Social Assistants',
    '35': 'Creative & Media','41': 'Clerical Support','42': 'Customer Services',
    '43': 'Data & Admin Support','51': 'Personal Services','52': 'Sales',
    '61': 'Agriculture','71': 'Construction','72': 'Manufacturing & Machinery',
    '74': 'Electronics & Installations','81': 'Plant/Machine Operators','83': 'Drivers & Operators',
    '91': 'Cleaners & Helpers','96': 'Labourers',
}

df = pd.read_csv("final_uni_studies.csv")
esco = pd.read_csv("/content/occupations_en.csv")

df['matched_label_clean'] = df['matched_label'].fillna('').astype(str).str.strip()
esco['preferredLabel_clean'] = esco['preferredLabel'].fillna('').astype(str).str.strip()

esco_map = esco.set_index('preferredLabel_clean')

df_exact = df[df['matched_label_clean'].isin(esco_map.index)].copy()

df_exact = df_exact.join(esco_map[['iscoGroup']], on='matched_label_clean', how='left', rsuffix='_esco')

df_exact['isco_major'] = df_exact['iscoGroup'].astype(str).str[:2]
df_exact['sector'] = df_exact['isco_major'].map(isco_to_sector)

df_exact.to_csv("fine_sector.csv", index=False)
print(f"Saved {len(df_exact)} fine_sector.csv")

Saved 453394 fine_sector.csv


In [8]:
buckets = {
    "Tech / IT & Software": {
        "IT & Software",
        "Electronics & Installations",
    },
    "Healthcare": {
        "Healthcare",
        "Health Technicians",
    },
    "Engineering / Technicians": {
        "Science & Engineering",
        "Technicians & Associate Professions",
        "Manufacturing & Machinery",
        "Plant/Machine Operators",
        "Drivers & Operators",
    },
    "Public Sector / Admin": {
        "Public Administration",
        "Clerical Support",
        "Management",
    },
    "Creative / Media": {
        "Creative & Media",
    },
    "Education": {
        "Education & Training",
    },
    "Sales / Business / Support": {
        "Business & Administration",
        "Business & Sales Support",
        "Sales",
        "Customer Services",
        "Data & Admin Support",
        "Legal & Social Assistants",
        "Legal, Social & Cultural",
        "Personal Services",
    },
}

broad_map = {}
for broad_name, fine_set in buckets.items():
    for fine in fine_set:
        broad_map[fine] = broad_name

df_exact['broad_sector'] = df_exact['sector'].map(broad_map).fillna("Others / Unclassified")

print("\nBroad sector distribution:")
print(df_exact['broad_sector'].value_counts(dropna=False))

df_exact.to_csv("broad_sector.csv", index=False)
print(f"\nSaved {len(df_exact)} rows with broad_sector to broad_sector.csv")


Broad sector distribution:
broad_sector
Sales / Business / Support    229437
Others / Unclassified          89154
Education                      44120
Engineering / Technicians      39220
Public Sector / Admin          22503
Healthcare                     12976
Tech / IT & Software           12140
Creative / Media                3844
Name: count, dtype: int64

Saved 453394 rows with broad_sector to broad_sector.csv
